In [15]:
def revise(sentence):
    words = sentence.split()
    result = []
    for word in words:
        tmp = word[0] + ''.join([word[i] for i in range(1, len(word)) if word[i] != word[i - 1]])
        if tmp == '스로':
            tmp = '스스로'
        result.append(tmp)
    return ' '.join(result)

In [16]:
revise('숙문을 취소하고  싶은데  가게  사장님 높서  전화를로 안안  받아요.')

'숙문을 취소하고 싶은데 가게 사장님 높서 전화를로 안 받아요.'

In [1]:
import torch
import torchaudio
import numpy as np
import librosa
from kospeech.vocabs.ksponspeech import KsponSpeechVocabulary

class STT:
    device = "cpu"
    model_path = "./STT_model.pt"
    vocab = KsponSpeechVocabulary('./aihub_character_vocabs.csv')
    model = torch.load(model_path, map_location=lambda storage, _: storage).module.to(device)

    def parse(signal):
        feature = torchaudio.compliance.kaldi.fbank(
            waveform=torch.tensor(signal).unsqueeze(0),
            num_mel_bins=80,
            frame_length=20,
            frame_shift=10,
            window_type='hamming'
        ).transpose(0, 1).numpy()
        feature -= feature.mean()
        feature /= np.std(feature)
        return torch.FloatTensor(feature).transpose(0, 1)

    def revise(sentence):
        words = sentence.split()
        result = []
        for word in words:
            tmp = word[0] + ''.join([word[i] for i in range(1, len(word)) if word[i] != word[i - 1]])
            if tmp == '스로':
                tmp = '스스로'
            result.append(tmp)
        return ' '.join(result)

    def __new__(cls, path):
        wav_data, sr = librosa.load(path, sr=16000)
        feature = cls.parse(wav_data)
        input_length = torch.LongTensor([len(feature)])

        y_hats = cls.model.recognize(feature.unsqueeze(0), input_length)
        sentence = cls.vocab.label_to_string(y_hats.cpu().detach().numpy())[0]
        return cls.revise(sentence)

d:\Coding\lincall-ai\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Coding\lincall-ai\.venv\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [ ]:
이거 배달이 잘못 온 거 같아요.

In [5]:
STT('G:\내 드라이브\lincall-ai\\audio0.wav')

'이게 배달이 잘못둔 거 같아 영.'

In [1]:
import speech_recognition as sr

class STT:
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300

    def __new__(cls, path):
        korean_audio = sr.AudioFile(path)

        with korean_audio as source:
            data = cls.recognizer.record(source)

        return cls.recognizer.recognize_google(audio_data=data, language="ko-KR")

In [3]:
STT('G:\내 드라이브\lincall-ai\\audio0.wav')

'배달이 잘못 온 거 같아요'